# Caso de Estudio A/B Testing
## *Probando la efectividad de una nueva característica de UX en una página web*

### **Autor:** Andrés Ulloa Jaramillo

![](https://useinsider.com/assets/media/2021/02/your-guide-to-ab-testing-4.png)

## Importar librerías:

In [ ]:
# Importar librerías:
import pandas as pd
import numpy as np
from scipy.stats import norm
from math import sqrt
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Configuración inicial:
np.set_printoptions(suppress=True)
pd.set_option("display.float_format", "{:.6f}".format)

## Metodología:

1. Introducción y planteamiento de hipótesis
2. Carga y exploración de datos
3. Validar supuestos
4. Prueba estadística
5. Conclusiones y recomendaciones

## **1. Introducción y planteamiento de hipótesis**

**Contexto del negocio:** Una empresa que cuenta con una tienda en línea desea implementar una nueva característica de UX dentro de la sección de compra de sus productos, con el fin de incrementar la participación del usuario, medida en clics. La versión actual o de control del botón de la página web dentro de la sección de compra supone un rectangulo con el texto "BUY NOW" situado debajo de la descripción del producto y su precio; mientras que la versión experimental supone un botón circular con el logo de un carro de compra ubicado en la esquina inferior derecha de la imagen del producto. De este modo, la empresa desea probar esta nueva característica mediante una prueba dividida o prueba A/B.

- **Botón en Versión de Control:** Botón rectangular con texto "BUY NOW", de dimensiones 585px (ancho) x 184px (alto), ubicado debajo de la descripción y precio del producto.
- **Botón en Versión Experimental:** Botón circular con logo carro de compra, de dimensiones 255px (ancho) x 255px (alto), ubicado sobrepuesto en la esquina inferior derecha de la imagen del producto.

**Objetivo principal:** Comparar dos versiones de un botón de compra en la tienda en línea de un negocio; donde varía la ubicación, forma, tamaño y texto o logo de dicho botón; mientras todo lo demás permanece igual. En base a ello, se quiere determinar si la diferencia en las tasas de clics entre los grupos experimentales y de control es estadísticamente significativa.

**Planteamiento de hipótesis:**
- **Hipótesis Nula (H0):** No hay diferencia significativa en la tasa de clics entre el grupo de control y el grupo experimental.
- **Hipótesis Alternativa (H1):** Sí hay una diferencia significativa en la tasa de clics entre el grupo de control y el grupo experimental.

**Parámetros de potencia definidos:**
- **Poder de la prueba:** **80%.** Significa que se acepta un 80% de probabilidad de detectar una diferencia significativa y un 20% de probabilidad de cometer un falso negativo; es decir, no detectar un efecto real.
- **Nivel de significancia:** **5%.** Supone un riesgo del 5% de detectar un falso positivo; es decir, concluir que existe una diferencia estadísticamente significativa entre los grupos cuando en realidad no la hay.
- **Efecto mínimo detectable** **10%.** Se trata de la diferencia mínima entre las tasas de clics de los dos grupos que se desea detectar para tomar una decisión.

## **2. Carga y exploración de datos**

In [ ]:
# Cargar el conjunto de datos
df = pd.read_csv("/kaggle/input/lunar-tech-case-study-ab-testing/ab_test_click_data (1).csv")

In [ ]:
# Visualizar las primeras filas del conjunto de datos
df.head()

In [ ]:
# Visualizar las últimas filas del conjunto de datos
df.tail()

In [ ]:
# Visualizar la información básica del conjunto de datos
df.info()

Podemos observar que el conjunto de datos cuenta con las siguientes variables:
- user_id: Identificador único del usuario.
- click: Variable binaria que indica si el usuario hizo clic en el botón (1) o no (0).
- group: Variable categórica que indica a qué grupo pertenece el usuario (control o test).
- timestamp: Variable que indica la fecha y hora exacta a la que se realizó la interacción del usuario en cuestión.

Además, podemos apreciar que el conjunto de datos tiene 20.000 registros o filas y 4 columnas. La variable de timestamp tiene 10.000 valores faltantes, parece que la fecha y hora de interacción no se registro para los usuarios pertenecientes al grupo de control; no obstante, esta no es una información relevante para los propósitos del presente estudio.

In [ ]:
# Suma de clics por grupo:
df.groupby("group")["click"].sum()

In [ ]:
# Visualización de la distribución de clics por grupos
# Asignar paleta de colores
palette = {0: 'black', 1: 'orange'}

# Graficar
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='group', hue='click', data=df, palette=palette)
plt.title("Distribución de clics en los grupos de control y experimental")
plt.xlabel("Grupo")
plt.ylabel("Conteo de clics")
plt.legend(title="Clic", labels=["No", "Sí"])  # Cambiar el argumento 'label' a 'labels'

# Calcular los porcentajes
group_counts = df.groupby(["group"]).size()
group_click_counts = df.groupby(["group", "click"]).size().reset_index(name="count")

for p in ax.patches:
    height = p.get_height()
    # Encontrar el grupo y el tipo de click
    group = "exp" if p.get_x() < 0.5 else "con"
    click = 1 if p.get_x() > 0.5 else 0
    total = group_counts[group]
    percentage = height / total * 100
    ax.text(
        p.get_x() + p.get_width() / 2,
        height + 5,
        f'{percentage:.1f}%',
        ha="center",
        color="black",
        fontsize=10,
    )

plt.tight_layout()
plt.show()

Los resultados sugieren que el número de clics es significativamente mayor en el grupo experimental en comparación con el grupo de control. Esto nos indica que la versión experimental del botón (circular con el logo) parece ser más efectiva en atraer clics que la versión de control (rectangular con texto).

Sin embargo, se desean conclusiones verificadas basadas en datos. Por lo que es necesario seguir los procedimientos estadísticos pertinentes para validar formalmente conclusiones sobre los resultados.

## **3. Validar supuestos**

Considerando las características del presente caso, se empleará una prueba paramétrica Z de dos muestras para comparar proporciones. Donde se deben verificar los siguientes supuestos:

- **Prueba de normalidad:** El tamaño de la muestra es maor a 30, por lo que puede usar el teorema del límite central para afirmar que las distribuciones de muestreo de los grupos de control y experimentales son asintóticamente normales.
- **Independencia de las observaciones:** Se garantizó aleatorización y un diseño robusto, por lo que se evitó cualquier tipo de sesgos entre usuarios.
- **Independencia entre grupos:** Se garantizó que los grupos no se influyan, por medio de una asignación aleatoria.
- **Datos binarios:** Las proporciones solo aplican a datos binarios. Se confirma valores binarios en el conjunto de datos presente (0/1).

## **4. Prueba estadística:**

Definimos los parámetros de potencia previamente establecidos:

In [ ]:
# Parámetros de potencia:
alpha = 0.05 #5% de nivel de significancia.
delta = 0.1 #Deseamos al menos un resultado 10% más alto (favorable) para el grupo de experimentación comparado con el grupo de control.

Es importante resaltar que primero queremos confirmar que existe una diferencia estadísticamente significativa entre la tasa de clics del grupo de control y el grupo de experimentación. Después, se revisa si esa diferencia es de al menos un 10% más alta (efecto mínimo detectable), favorable para el grupo de experimentación, en comparación con el grupo de control.

Partimos por separar los datos en grupos para facilitar posteriores cálculos:

In [ ]:
# Separar los datos en grupos
con_group = df[df['group'] == 'con'] #Grupo de control
exp_group = df[df['group'] == 'exp'] #Grupo experimental

Calculamos el tamaño de muestra para cada grupo:

In [ ]:
# Calcular el tamaño de muestra por grupo
n_con = len(con_group)
n_exp = len(exp_group)

print("Tamaño de muestra para el grupo de control: ", n_con)
print("Tamaño de muestra para el grupo experimental: ", n_exp)

Calculamos el número de clics por grupo:

In [ ]:
# Calcular el número de clics por grupo
x_con = con_group["click"].sum()
x_exp = exp_group["click"].sum()

print("Número de clics en grupo de control: ", x_con)
print("Número de clics en grupo experimental: ", x_exp)

Calculamos las estimaciones de probabilidad de clic para cada grupo:

In [ ]:
# Calcular la probabilidad de clic por grupo
p_con = con_group['click'].mean()
p_exp = exp_group['click'].mean()

print("Probabilidad de clic para el grupo de control: ", p_con)
print("Probabilidad de clic para el grupo experimental: ", p_exp)

Necesitamos obtener una estimación de la probabilidad agrupada de éxito y una estimación de la varianza agrupada, para facilitar el cálculo del error estándar:

In [ ]:
# Calcular la probabilidad acumulada de clics
p_agrupada = (x_con+x_exp)/(n_con+n_exp)
var_agrupada = p_agrupada * (1-p_agrupada) * (1/n_con + 1/n_exp)

print(f"Probabilidad acumulada de clics: {p_agrupada:.5f}")
print(f"Varianza acumulada: {var_agrupada:.6f}")

Con ello, calculamos el error estándar:

In [ ]:
# Calcular el error estándar
SE = np.sqrt(var_agrupada)
print("El Error Estándar es: ", SE)

En consecuencia, el estadístico de prueba de la prueba Z de 2 muestras para la diferencia de proporciones se pueden calcular de la siguiente manera:

In [ ]:
# Calcular el estadístico de prueba Z
Test_stat = (p_con - p_exp)/SE
print("El estadístico de prueba Z de 2 muestras es: ", Test_stat)

Y, el valor crítico de Z se calcula a continuación:

In [ ]:
# Calcular el valor Z
Z_crit = norm.ppf(1-alpha/2)
print("El valor crítico de Z de la distribución normal estándar: ", Z_crit)

In [ ]:
# Parámetros de la distribución normal estándar
mu = 0  # Media para una distribución normal
sigma = 1  # Desviación estándar
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
y = norm.pdf(x, mu, sigma)

# Graficar la distribución normal estándar
plt.plot(x, y, label='Distribución normal estándar')

# Sombrear las regiones de rechazo para una prueba de dos colas
plt.fill_between(x, y, where=(x > Z_crit) | (x < -Z_crit), color='red', alpha=0.5, label='Región de rechazo')

# Añadir estadístico de prueba
plt.axvline(Test_stat, color='green', linestyle='dashed', linewidth=2, label=f'Estadístico de prueba = {Test_stat:.2f}')

# Añadir valor crítico de Z
plt.axvline(Z_crit, color='blue', linestyle='dashed', linewidth=1, label=f'Z-crítico = {Z_crit:.2f}')
plt.axvline(-Z_crit, color='blue', linestyle='dashed', linewidth=1)

# Añadir título y etiquetas
plt.xlabel('Valor Z')
plt.ylabel('Densidad de probabilidad')
plt.title('Distribución gaussiana con región de rechazo')
plt.legend()

plt.show()

Claramente el estadístico de prueba se encuentra en la región de rechazo de la hipótesis nula.

Como complemento y para reforzar las conclusiones sobre los resultados, emplearemos el cálculo del valor P, para validar los resultados y tomar una decisión acertada sobre la hipótesis.

**Calcular el valor P:**

Indicaciones para interpretar los resultados del valor P:
- Un valor de P bajo (p <= 0.05 a un nivel de significancia del 5%) nos sugiere que hay suficiente evidencia para rechazar la hipótesis nula.
- Un valor de P elevado (p >= 0.05 a un nivel de significancia del 5%) nos sugiere que NO hay suficiente evidencia como para rechazar la hipótesis nula; así que no la rechazamos.

In [ ]:
# Calcular el valor P
p_value = 2 * norm.sf(abs(Test_stat))
print("Valor P: ",round(p_value,6))

Un valor P de 0.0 es menor al nivel de significancia de 0.05; por lo que tenemos suficiente evidencia para rechazar la hipótesis nula con un nivel de confianza del 95%. **Esto nos sugiere que SÍ existe una diferencia estadísticamente significativa para la tasa de clicks entre el grupo de control y el grupo de experimentación. Es decir, sí hay un efecto real, formalmente comprobado.**

Ahora bien, procederemos a calcular el intervalo de confianza para determinar la diferencia real entre las proporciones de clics del grupo experimental y el grupo de control. Además, con esto podemos comprobar si existe una significancia práctica.

**Calcular el intervalo de confianza:**

In [ ]:
# Calcular el intervalo de confianza
CI = [round((p_exp - p_con) - SE*Z_crit,3), round((p_exp - p_con) + SE*Z_crit,3)]
print("Intervalo de Confianza: ", CI)

El intervalo de confianza se calcula alrededor de la diferencia observada entre las proporciones de los dos grupos (p_exp - p_con). En este sentido, si el intervalo esta por encima de 0, se respalda la idea de que el grupo experimental tiene una proporción mayor de clics en comparación con el grupo de control.

El intervalo de confianza resultante sugiere que, con un 95% de confianza, la diferencia real entre las proporciones de clics del grupo experimental y el grupo de control está entre 0.399 (39.9%) y 0.426 (42.6%).

Vemos que se trata de un rango estrecho, lo que sugiere que podemos tener confianza en los resultados obtenidos. Ahora bien, ¿significa esto que tenemos significancia práctica?. Pues, sabiendo que el nivel mínimo detectable fue establecido por la empresa en un 0.1; entonces un intervalo de confianza entre 0.39 y 0.42 nos permite decir que el grupo experimental tiene una proporción considerablemente mayor de clics en comparación con el grupo de control y **SÍ tenemos significancia práctica**.

## **5. Conclusiones y recomendaciones:**

**Conclusiones:**

1. Diferencia estadísticamente significativa:
   - El análisis de la prueba Z de dos muestras mostró que el estadístico de prueba (cae en la región de rechazo de la hipótesis nula.
   - El valor p es 0.0, que es menor al nivel de significancia del 5%; indica que existe suficiente evidencia para rechazar la hipótesis nula y concluir que hay una diferencia estadísticamente significativa en las tasas de clics entre los grupos experimental y de control.


2. Intervalo de confianza:

   - El intervalo de confianza del 95% para la diferencia entre las proporciones de clics está entre 39.9% y 42.6%.
   - Este intervalo está completamente por encima de 0, lo que confirma que el grupo experimental tiene una mayor tasa de clics que el grupo de control.


3. Significancia práctica:

   - El efecto mínimo detectable (MDE) establecido por la empresa fue del 10%. La diferencia observada y validada (intervalo de confianza entre 39.9% y 42.6%) supera con creces este umbral, lo que indica que el grupo experimental no solo es significativamente mejor, sino que también tiene relevancia práctica para el negocio.

4. Efectividad del botón experimental:

   - La tasa de clic promedio del grupo experimental (61.16%) es considerablemente mayor que la del grupo de control ( 19.89%). Esto sugiere que el nuevo diseño del botón (circular con un carrito de compras) es mucho más efectivo en captar la atención de los usuarios.

**Recomendaciones:**
1. Implementar el diseño experimental del botón dado el análisis estadístico y práctico que respalda el exito de esta implementación.
2. Monitorear resultados tras la implementación, evaluando el impacto en métricas clave.
3. Explorar posibles mejoras adicionales.
4. Considerar extender el análisis a métricas secundarias, como el número total de compras o el valor promedio de los pedidos.